In [30]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Cleaned Datasets/premier_league_features_df.csv'
path_2 = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Cleaned Datasets/combined_elo_with_features_df.csv'


premier_league_features_df = pd.read_csv(path)
combined_elo_with_features_df = pd.read_csv(path_2)

missing_information_list= ['segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2020.0',
 'segunda_liga: 2012.0',
 'segunda_liga: 2012.0']

In [29]:
# Function to remove partially completed seasons

def _remove_incomplete_seasons(missing_information_list, combined_elo_with_features_df):

    missing_info_set = set(missing_information_list)
    missing_info_deduplicated_list = list(missing_info_set)
    missing_info_deduplicated_list
    for item in missing_info_deduplicated_list:
        league = item.split(": ")[0]
        season = item.split(": ")[1]
        combined_elo_with_features_df.drop(combined_elo_with_features_df.index[(combined_elo_with_features_df.League == league)&(
                        combined_elo_with_features_df.Season == season)], inplace=True)
    
    return combined_elo_with_features_df

def _prepare_dataset_for_ML_models(df):


    df.drop(axis=1, columns=['Result','Round', 'Link', 'Home_Win', 'Away_Win', 'Date_New', 'Home_Team', 'Away_Team', 'Season', 'League', 'Referee', 'City', 'Country', 'Stadium', 'Pitch','Home_Yellow', 'Home_Red', 'Away_Yellow', 'Away_Red'], inplace=True)
    df["Goal_Difference"] = df["Home_Goals"] - df["Away_Goals"]
    df.drop(axis=1, columns=['Home_Goals', 'Away_Goals'], inplace=True)
    
    return df



In [31]:
combined_elo_with_partial_seasons_removed_df = _remove_incomplete_seasons(missing_information_list, combined_elo_with_features_df)
combined_elo_numerical_df = _prepare_dataset_for_ML_models(combined_elo_with_partial_seasons_removed_df)

combined_elo_numerical_df

,Capacity,ELO_home,ELO_away,home_team_total_goals_scored_so_far,home_team_total_goals_conceeded_so_far,home_team_current_win_streak,home_team_current_loss_streak,home_team_total_points_so_far,home_team_current_goal_drought,home_team_total_wins_so_far,away_team_total_goals_scored_so_far,away_team_total_goals_conceeded_so_far,away_team_current_win_streak,away_team_current_loss_streak,away_team_total_points_so_far,away_team_current_goal_drought,away_team_total_wins_so_far,Goal_Difference
0,19920.0,65.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,19920.0,65.0,50.0,1.0,0.0,1.0,0.0,3.0,0.0,1.0,3.0,1.0,2.0,0.0,6.0,0.0,2.0,1.0
2,19920.0,64.0,58.0,3.0,1.0,1.0,0.0,9.0,0.0,3.0,11.0,4.0,0.0,1.0,7.0,2.0,2.0,2.0
3,19920.0,64.0,63.0,6.0,2.0,2.0,0.0,12.0,0.0,4.0,9.0,5.0,0.0,0.0,12.0,0.0,3.0,0.0
4,19920.0,64.0,66.0,8.0,5.0,0.0,1.0,14.0,1.0,4.0,10.0,9.0,1.0,0.0,15.0,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122144,7220.0,52.0,59.0,16.0,32.0,0.0,2.0,27.0,0.0,7.0,25.0,30.0,2.0,0.0,29.0,0.0,6.0,1.0
122145,7220.0,52.0,52.0,18.0,35.0,0.0,1.0,30.0,0.0,8.0,30.0,31.0,1.0,0.0,37.0,0.0,9.0,1.0
122146,7220.0,52.0,52.0,22.0,36.0,2.0,0.0,36.0,0.0,10.0,36.0,35.0,1.0,0.0,38.0,0.0,9.0,0.0
122147,7220.0,52.0,63.0,23.0,37.0,0.0,0.0,37.0,0.0,10.0,40.0,37.0,1.0,0.0,46.0,0.0,12.0,1.0


In [2]:
from sqlalchemy import create_engine, inspect
from decouple import config


def _connect_to_RDS() -> create_engine:
    '''_connect_to_RDS 
    Method to craete connection to RDS database using sqlalchemy

    Returns:
        engine - connection

    '''
    DATABASE_TYPE = config('RDS_DATABASE_TYPE')
    DBAPI = config('RDS_DBAPI')
    ENDPOINT = config('RDS_ENDPOINT')
    USER = config('RDS_USER')
    PASSWORD = config('RDS_DATABASE_PASSWORD')
    PORT = 5432
    DATABASE = config('RDS_DATABASE')

    engine = create_engine(
        f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
    return engine

def _upload_dataframes_to_rds(df):
    '''_upload_dataframes_to_rds 
    If dataframe exists, append it to SQL database.
    '''

    engine = _connect_to_RDS()
    df.to_sql('players', engine, if_exists='append')

   



In [32]:
from decouple import config

DATABASE_TYPE = config('RDS_DATABASE_TYPE')

UndefinedValueError: RDS_DATABASE_TYPE not found. Declare it as envvar or define a default value.

In [ ]:
_upload_dataframes_to_rds(premier_league_features_df)